In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.utils import resample
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn import model_selection
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from scipy.stats import zscore
import matplotlib.pyplot as plt 

In [2]:
df = pd.read_csv("housing.csv",index_col=0)  
df.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [3]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [4]:
import statsmodels.formula.api as smf

In [5]:
m1=smf.ols('medv~crim+zn+chas+indus+age+nox+rm+dis+rad+tax+ptratio+black+lstat',df).fit()

In [6]:
m1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   medv   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     108.1
Date:                Thu, 11 Jul 2019   Prob (F-statistic):          6.72e-135
Time:                        12:06:31   Log-Likelihood:                -1498.8
No. Observations:                 506   AIC:                             3026.
Df Residuals:                     492   BIC:                             3085.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     36.4595      5.103      7.144      0.000      26.432      46.487
crim          -0.1080      0.033     -3.287      0.001      -0.173      -0.043
zn             0.0464      0.014      3.382      0.001       0.019       0.073
chas           2.6867      0.862      3.118      0.002       0.994       4.380
indus          0.0206      0.061      0.334      0.738      -0.100       0.141
age            0.0007      0.013      0.052      0.958      -0.025       0.027
nox          -17.7666      3.820     -4.651      0.000     -25.272     -10.262
rm             3.8099      0.418      9.116      0.000       2.989       4.631
dis           -1.4756      0.199     -7.398      0.000      -1.867      -1.084
rad            0.3060      0.066      4.613      0.000       0.176       0.436
tax           -0.0123      0.004     -3.280      0.001      -0.020      -0.005
ptratio       -0.9527      0.131     -7.283      0.000      -1.210      -0.696
black          0.0093      0.003      3.467      0.001       0.004       0.015
lstat         -0.5248      0.051    -10.347      0.000      -0.624      -0.425
==============================================================================
Omnibus:                      178.041   Durbin-Watson:                   1.078
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              783.126
Skew:                           1.521   Prob(JB):                    8.84e-171
Kurtosis:                       8.281   Cond. No.                     1.51e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.51e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [7]:
X=df.drop(['medv','indus','age'],axis=1)
Y=df[['medv']]

In [8]:
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression

In [16]:
type(X)

pandas.core.frame.DataFrame

In [17]:
type(Y)

pandas.core.frame.DataFrame

In [9]:
LR=LinearRegression()

In [23]:
gboost=GradientBoostingRegressor(n_estimators=120,random_state=0)

In [24]:
boost_dt=AdaBoostRegressor(n_estimators=60,random_state=0)

In [25]:
boost_LR=AdaBoostRegressor(base_estimator=LR,n_estimators = 60,random_state=0)

In [26]:
bag_dt=BaggingRegressor(n_estimators=15,random_state=0)

In [27]:
bag_LR=BaggingRegressor(base_estimator=LR,n_estimators=15,random_state=0)

In [29]:
from sklearn.model_selection import KFold
from sklearn import metrics
kf=KFold(n_splits=5,shuffle=True,random_state=2)
for model, name in zip([LR,gboost,boost_dt,boost_LR,bag_dt,bag_LR],
        ['LinearReg','GradientBoost','BoostedDT','BoostedLR','BaggedDT','BaggedLR']):
    rmse=[]
    for train,test in kf.split(X,Y):
        Xtrain,Xtest=X.iloc[train,:],X.iloc[test,:]
        Ytrain,Ytest=Y.iloc[train],Y.iloc[test]
        model.fit(Xtrain,Ytrain)
        Y_predict=model.predict(Xtest)
        #mse=np.sum((Ytest-Y_predict)**2)/len(Ytest)
        mse=metrics.mean_squared_error(Ytest,Y_predict)
        rmse.append(np.sqrt(mse))
    print(rmse)
    print("RMSE scores: %0.02f (+/- %0.5f) [%s]" % (np.mean(rmse), np.var(rmse,ddof=1), name ))   

[4.2724150578441416, 5.321458766441966, 5.58473217836811, 4.964536520477553, 3.667686565726027]
RMSE scores: 4.76 (+/- 0.61703) [LinearReg]
[2.7991496608688764, 2.7873280146054045, 2.938566532042719, 3.4774965389516574, 2.5146835876815]

C:\Users\GURURAJAN\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\GURURAJAN\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\GURURAJAN\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\GURURAJAN\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to


RMSE scores: 2.90 (+/- 0.12657) [GradientBoost]


C:\Users\GURURAJAN\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\GURURAJAN\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\GURURAJAN\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\GURURAJAN\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to

[3.2302718174653053, 4.62531019186168, 3.9186267656847495, 3.686093666077418, 3.614806828064498]
RMSE scores: 3.82 (+/- 0.26649) [BoostedDT]
[5.081948553637034, 5.30859238496251, 5.710926972410301, 5.437352820631648, 4.040230327390831]
RMSE scores: 5.12 (+/- 0.41318) [BoostedLR]
[3.183169177576318, 3.632196933738112, 3.3700514958541024, 4.1448398871089065, 2.732869074313245]
RMSE scores: 3.41 (+/- 0.27522) [BaggedDT]
[4.217563967870641, 5.346334066243864, 5.554697289724123, 4.932883589957159, 3.6416644299545897]
RMSE scores: 4.74 (+/- 0.63696) [BaggedLR]


C:\Users\GURURAJAN\Anaconda3\lib\site-packages\sklearn\ensemble\bagging.py:397: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)
C:\Users\GURURAJAN\Anaconda3\lib\site-packages\sklearn\ensemble\bagging.py:397: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)
C:\Users\GURURAJAN\Anaconda3\lib\site-packages\sklearn\ensemble\bagging.py:397: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)
C:\Users\GURURAJAN\Anaconda3\lib\site-packages\sklearn\ensemble\bagging.py:397: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shap

In [21]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

RMSE scores: 4.76 (+/- 0.61703) [LinearReg]
RMSE scores: 2.90 (+/- 0.12657) [GradientBoost]
RMSE scores: 3.41 (+/- 0.27522) [BaggedDT]
RMSE scores: 4.74 (+/- 0.63696) [BaggedLR]
RMSE scores: 3.82 (+/- 0.26649) [BoostedDT]
RMSE scores: 5.12 (+/- 0.41318) [BoostedLR]    